In [5]:
import tabulate
import operator
import os
import pandas as pd
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents.agent_types import AgentType
from dotenv import load_dotenv, dotenv_values 

load_dotenv()

True

In [6]:
openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] ="multi-agent"
os.environ['OPENAI_API_KEY'] = openai_api_key

In [8]:
file_path= 'data.csv'

csv_agent= create_csv_agent(
        ChatOpenAI(temperature=0, model="gpt-4"),
        file_path, 
        verbose=True,
        stop=["\nObservation:"],
        agent_type=AgentType.OPENAI_FUNCTIONS,
        handle_parsing_errors=True
    )
    
csv_agent.run("how mant diffrent countries are there? ")

c:\SmartCart_MBA\streamlit_app\venv\lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:242: UserWarning: Received additional kwargs {'stop': ['\nObservation:'], 'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Country'].nunique()"}`


38There are 38 different countries in the dataframe.

> Finished chain.


'There are 38 different countries in the dataframe.'

In [11]:
csv_agent.run("Which product generates more revenue?")




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Revenue'] = df['Price'] * df['Quantity']"}`
responded: To find out which product generates more revenue, we need to create a new column in the dataframe that represents the total revenue for each transaction. This can be calculated by multiplying the Price and Quantity columns. Then, we can group by the ProductName and sum the revenue, sorting in descending order to find the product that generates the most revenue.

Let's do this in steps. First, let's create the new column 'Revenue'.


Invoking: `python_repl_ast` with `{'query': "df.groupby('ProductName')['Revenue'].sum().sort_values(ascending=False).head(1)"}`
responded: Now that we have the 'Revenue' column, let's group by 'ProductName' and sum the 'Revenue', then sort in descending order to find the product that generates the most revenue.

ProductName
Popcorn Holder    583286.51
Name: Revenue, dtype: float64The product that generates the most

"The product that generates the most revenue is the 'Popcorn Holder' with a total revenue of 583286.51."

In [12]:
question = "Give me the user id and other details of the custmer who spent the most"

propmt = """ You are a question-answering bot over the data you queried from dataframes.
           Give responses to the question on the end of the text :
            
            
            Given a dataset from an e-commerce platform detailing sales transactions over a year, analyze the data to understand consumer behavior and sales patterns. This dataset originates from a UK-based online retail shop specializing in gifts and homewares for both adults and children, operating since 2007. With customers worldwide, the business model includes direct consumer sales and bulk purchases by small businesses for resale.

            Dataset Description:

            - TransactionNo (categorical): A unique six-digit number for each transaction. Transactions beginning with "C" denote cancellations.
            - Date (numeric): The date of each transaction.
            - ProductNo (categorical): A unique identifier for each product, consisting of five or six digits.
            - Product (categorical): The name of the product/item.
            - Price (numeric): The price per unit of each product in pound sterling (£).
            - Quantity (numeric): The number of units per product in each transaction. Negative quantities indicate cancelled transactions.
            - CustomerNo (categorical): A unique five-digit number identifying each customer.
            - Country (categorical): The customer's country of residence.

            A notable aspect of this dataset is the presence of order cancellations, often due to products being out of stock. Customers generally prefer all items in their order to be delivered simultaneously, leading to cancellations when this isn't possible.

            
            Below is the question
            Question:"""  + question

In [15]:
csv_agent.run(propmt)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['TotalSpent'] = df['Price'] * df['Quantity']\ndf.groupby('CustomerNo')['TotalSpent'].sum().idxmax()"}`


14646.0
Invoking: `python_repl_ast` with `{'query': "df[df['CustomerNo'] == 14646]"}`


       TransactionNo        Date ProductNo                        ProductName  \
4024          581338   12/8/2019     23344           Jumbo Bag 50'S Christmas   
8486          581175   12/7/2019     22973       Children's Circus Parade Mug   
8487          581175   12/7/2019     23280  Folding Butterfly Mirror Hot Pink   
8488          581175   12/7/2019     23281       Folding Butterfly Mirror Red   
8489          581175   12/7/2019     23498              Classic Bicycle Clips   
...              ...         ...       ...                                ...   
497157        539491  12/20/2018    47599A                    Pink Party Bags   
497158        539491  12/20/2018     22331   Woodland Party Bag + Stic

"The customer who spent the most is the one with the CustomerNo 14646. This customer is from the Netherlands. They have made 2066 transactions. Some of the products they have purchased include 'Jumbo Bag 50'S Christmas', 'Children's Circus Parade Mug', 'Folding Butterfly Mirror Hot Pink', 'Folding Butterfly Mirror Red', and 'Classic Bicycle Clips'."

In [16]:
output = csv_agent("How many rows?")

c:\SmartCart_MBA\streamlit_app\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


536350The dataframe has 536,350 rows.

> Finished chain.


In [19]:
output['output']

'The dataframe has 536,350 rows.'

## Lets build a fastapi for the above functionality

In [20]:
# from typing import Union

# from fastapi import FastAPI
# from pydantic import BaseModel

# class InputQuery(BaseModel):
#     query: str

# app = FastAPI()

# @app.put("/csv_agent/")
# def csv_agent_api(query: InputQuery):
#     output = csv_agent(query.query)
#     return {'Query': query.query, 'Output': output['output']}


## Refer csv_agent_api.py file